# Vendor recommender - Collaborative Filtering (CF) outcomes
@olibolly

- If a person A likes item 1, 2, 3 and B like 2,3,4 then they have similar interests and A should like item 4 and B should like item 1.
- Looking at match between gov mod_agency (#275) & vendors (#770526) & product (#)
- Reference: https://cambridgespark.com/content/tutorials/implementing-your-own-recommender-systems-in-Python/index.html

<br>

### OUTCOMES

Outcomes are stored as big query tables with the following naming:

1. Agency & Vendor
    * vendor_vendor_by_agency *(Item-Item CF)*
    * agency_vendor *(User-Item CF)*
    * vendor_vendor_by_agency_normalized
    * agency_vendor_normalized
    * vendor_vendor_by_agency_smb
    * agency_vendor_smb
2. Product & Vendor
    * vendor_vendor_by_product *(Item-Item CF)*
    * product-vendor *(User-Item CF)*
    * vendor_vendor_by_product_normalized
    * product-vendor_normalized
    * vendor_vendor_by_product_smb
    * product-vendor_smb

<br>

### TO DO
- **TRAINING (1.9M rows) kernel crashed abover 20K -> Need to Map/Reduce or getting a higher performance machine or use another algorithm (matrix factorization)?**
- **Think about scaling or binarizing the count data -> to improve results**
- **Only run on SMBs**
- **Look at match between product service code (#5833) & vendors (#770526) **
- **add FILTER: Geo**
- **add FILTER: Already done business with a company?**
- **Save outcome in output tables for front end Big query API access**
- **Join with SAM and FAPIIS**
- **Test on contractingofficeid**

In [3]:
import google.datalab.bigquery as bq
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt

from datalab.context import Context
import datalab.storage as storage

## 1. Agency & Vendor

In [6]:
%%bq query -n df_query
select
contractingofficerbusinesssizedetermination,
mod_agency,
vendorname,
count(*) as count
from `fiery-set-171213.vrec.usa_spending_all`
where vendorcountrycode in ('UNITED STATES', 'USA: UNITED STATES OF AMERICA')
and contractingofficerbusinesssizedetermination in ('O: OTHER THAN SMALL BUSINESS', 'S: SMALL BUSINESS')
and mod_agency not in ("")
group by 1,2,3
order by count DESC
limit 10000

In [7]:
df = df_query.execute(output_options=bq.QueryOutput.dataframe()).result()
df.head()

,contractingofficerbusinesssizedetermination,mod_agency,vendorname,count
0,O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,AMERISOURCEBERGEN DRUG CORPORA,676166
1,O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,"CARDINAL HEALTH 200, LLC",419654
2,O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,"CARDINAL HEALTH, INC",318861
3,O: OTHER THAN SMALL BUSINESS,4730: FEDERAL ACQUISITION SERVICE,NATIONAL INDUSTRIES FOR THE BLIND,308991
4,O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,"OWENS & MINOR DISTRIBUTION, INC.",262205


In [8]:
df1 = df.drop('contractingofficerbusinesssizedetermination', axis = 1)
n_agency = df1.mod_agency.unique().shape[0]
n_vendors = df1.vendorname.unique().shape[0]
print 'Number of gov agency = ' + str(n_agency) + ' | Number of vendors = ' + str(n_vendors)

Number of gov agency = 107 | Number of vendors = 7357


In [9]:
# Convert categorial value with label encoding

le_agency = LabelEncoder()
label_agency = le_agency.fit_transform(df1['mod_agency'])

le_vendor = LabelEncoder()
label_vendor = le_vendor.fit_transform(df1['vendorname'])

df_agency = pd.DataFrame(label_agency)
df_vendor = pd.DataFrame(label_vendor)

df2 = pd.concat([df_agency, df_vendor], axis = 1)
df2 = pd.concat([df2, df1['count']], axis = 1)
df2.columns = ['mod_agency', 'vendorname', 'count']
df2.head(5)

# To ge the right label back
# le_agency.inverse_transform([173, 100])

,mod_agency,vendorname,count
0,98,514,676166
1,98,1320,419654
2,98,1328,318861
3,49,4648,308991
4,98,4990,262205


In [10]:
# Split into training and test data set
train_data, test_data = cv.train_test_split(df2, test_size=0.25)

In [11]:
#Build the matrix
train_data_matrix = np.zeros((n_agency, n_vendors))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_agency, n_vendors))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

#Compute cosine distance
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred
  
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [12]:
# Evaluation
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten() #filter out all items with no 0 as we only want to predict in the test set
    return sqrt(mean_squared_error(prediction, ground_truth))

print 'User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))
print 'Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix))

User-based CF RMSE: 11348.4140071
Item-based CF RMSE: 11393.1526963


In [13]:
print 'Worklow 1'
print '=' * 100
print 'Select your agency:'
agency = df1['mod_agency'][5]
print agency
print '=' * 100
print '1. Have you considered working with these SMB companies (user-item prediction)?'

agency = le_agency.transform(agency)
vendor_reco = pd.DataFrame(user_prediction[agency, :])
labels = pd.DataFrame(le_vendor.inverse_transform(range(0, len(vendor_reco))))

df_reco = pd.concat([vendor_reco, labels], axis = 1)
df_reco.columns = ['reco_score', 'vendorname']
#Join to get the SMB list
df_smb = df.drop(['mod_agency', 'count'], axis = 1)
df_reco = df_reco.set_index('vendorname').join(df_smb.set_index('vendorname'))

df_reco = df_reco.sort_values(['reco_score'], ascending = [0])
df_reco[df_reco['contractingofficerbusinesssizedetermination'] == 'S: SMALL BUSINESS'].head(10)

Worklow 1
Select your agency:
97AS: DEFENSE LOGISTICS AGENCY
1. Have you considered working with these SMB companies (user-item prediction)?


,reco_score,contractingofficerbusinesssizedetermination
vendorname,,
KAUFMAN COMPANY INC,1947.851863,S: SMALL BUSINESS
KAUFMAN COMPANY INC,1947.851863,S: SMALL BUSINESS
"KATMAI SUPPORT SERVICES, L.L.C.",1799.215087,S: SMALL BUSINESS
W W GRAINGER INCORPORATED (0280),1529.508847,S: SMALL BUSINESS
W W GRAINGER INCORPORATED (0280),1529.508847,S: SMALL BUSINESS
"BOH ENVIRONMENTAL, LLC",1447.632646,S: SMALL BUSINESS
FORD BRASADA LTD,1180.249700,S: SMALL BUSINESS
GENERAL ENVIRONMENTAL MANAGEMENT INCORPORATED,951.261063,S: SMALL BUSINESS
SIGVARIS INCORPORATED,755.874016,S: SMALL BUSINESS


In [14]:
print '=' * 100
print '2. Have you considered working with these SMB companies (item-item prediction?)'
vendor_reco = pd.DataFrame(item_prediction[agency, :])

df_reco = pd.concat([vendor_reco, labels], axis = 1)
df_reco.columns = ['reco_score', 'vendorname']
df_reco = df_reco.set_index('vendorname').join(df_smb.set_index('vendorname'))
df_reco = df_reco.sort_values(['reco_score'], ascending = [0])
df_reco[df_reco['contractingofficerbusinesssizedetermination'] == 'S: SMALL BUSINESS'].head(10)

2. Have you considered working with these SMB companies (item-item prediction?)


,reco_score,contractingofficerbusinesssizedetermination
vendorname,,
GILL GROUP INC,1.419972,S: SMALL BUSINESS
HAWKER PACIFIC AEROSPACE,1.410768,S: SMALL BUSINESS
B & E MANUFACTURING CO INC,1.400379,S: SMALL BUSINESS
"HOWELL LABORATORIES, INC.",1.396168,S: SMALL BUSINESS
"SYNTECH, INC.",1.393566,S: SMALL BUSINESS
WORLD WIDE TECHNOLOGY,1.392997,S: SMALL BUSINESS
WORLD WIDE TECHNOLOGY,1.392997,S: SMALL BUSINESS
WORLD WIDE TECHNOLOGY,1.392997,S: SMALL BUSINESS
WORLD WIDE TECHNOLOGY,1.392997,S: SMALL BUSINESS


In [15]:
print 'Worklow 2'
print '=' * 100
print 'Select a product code or enter a search query:'

# Workflow 2  - WIP
# Select a vendor
# Other similar vendor

Worklow 2
Select a product code or enter a search query:


## Writing outputs to Big Query

In [18]:
#Switch from matrix form to a long form
unique_label_agency = le_agency.inverse_transform(range(0, user_prediction.shape[0]))
unique_label_vendor = le_vendor.inverse_transform(range(0, user_prediction.shape[1]))

df_unique_label_agency = pd.DataFrame(unique_label_agency, columns = ['agency'])
df_unique_label_agency['agency_index'] = df_unique_label_agency.index

df_unique_label_vendor = pd.DataFrame(unique_label_vendor, columns = ['vendor'])
df_unique_label_vendor['vendor_index'] = df_unique_label_vendor.index

df_output = pd.DataFrame(data = user_prediction)
#df_output = pd.DataFrame(data = user_prediction, index = unique_label_agency , columns = unique_label_vendor)

df_output = df_output.unstack().reset_index()
df_output.columns = ['vendor_index', 'agency_index', 'reco_score']
df_output = df_output.set_index('agency_index').join(df_unique_label_agency.set_index('agency_index'))
df_output = df_output.set_index('vendor_index').join(df_unique_label_vendor.set_index('vendor_index'))

df_output = df_output.reset_index().drop(['vendor_index'], axis = 1)

df_output.head()

,reco_score,agency,vendor
0,-19.175728,1145: PEACE CORPS,"101MOBILITY, LLC"
1,-17.945543,"1205: USDA, OFFICE OF THE CHIEF FINANCIAL OFFICER","101MOBILITY, LLC"
2,-15.506435,1226: FOREST SERVICE,"101MOBILITY, LLC"
3,-18.952880,12C2: FOREST SERVICE,"101MOBILITY, LLC"
4,-18.227436,12C3: NATURAL RESOURCES CONSERVATION SERVICE,"101MOBILITY, LLC"


In [19]:
# Dataframe to write
#def write_to_big_query(df, bigquery_dataset_name = 'vrec', bigquery_table_name = 'test_table'):
    
#df = pd.DataFrame(data=[{1,2,3},{4,5,6}],columns=['a','b','c'])
df = df_output

bigquery_dataset_name = 'vrec'
bigquery_table_name = 'TestTable'

sample_bucket_name = Context.default().project_id + bigquery_dataset_name
sample_bucket_path = 'gs://' + sample_bucket_name
sample_bucket_object = sample_bucket_path + '/' + bigquery_table_name + '.txt'

# Define storage bucket
sample_bucket = storage.Bucket(sample_bucket_name)

# Create storage bucket if it does not exist
if not sample_bucket.exists():
    sample_bucket.create()

# Define BigQuery dataset and table
dataset = bq.Dataset(bigquery_dataset_name)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)

# Create BigQuery dataset
if not dataset.exists():
    dataset.create()

# Create or overwrite the existing table if it exists
table_schema = bq.Schema.from_data(df)
table.create(schema = table_schema, overwrite = True)

# Write the DataFrame to GCS (Google Cloud Storage)
%storage write --variable df --object $sample_bucket_object

# Write the DataFrame to a BigQuery table
table.insert(df)

Exception: insertAll failed: [{u'index': 0, u'errors': [{u'debugInfo': u'generic::not_found: no such field.', u'reason': u'invalid', u'message': u'no such field.', u'location': u'agency'}]}, {u'index': 1, u'errors': [{u'debugInfo': u'generic::not_found: no such field.', u'reason': u'invalid', u'message': u'no such field.', u'location': u'agency'}]}, {u'index': 2, u'errors': [{u'debugInfo': u'generic::not_found: no such field.', u'reason': u'invalid', u'message': u'no such field.', u'location': u'agency'}]}, {u'index': 3, u'errors': [{u'debugInfo': u'generic::not_found: no such field.', u'reason': u'invalid', u'message': u'no such field.', u'location': u'agency'}]}, {u'index': 4, u'errors': [{u'debugInfo': u'generic::not_found: no such field.', u'reason': u'invalid', u'message': u'no such field.', u'location': u'agency'}]}, {u'index': 5, u'errors': [{u'debugInfo': u'generic::not_found: no such field.', u'reason': u'invalid', u'message': u'no such field.', u'location': u'agency'}]}, {u'i

In [ ]:
simple_dataframe = pd.DataFrame(data=[{1,2,3},{4,5,6}],columns=['a','b','c'])
write_to_big_query(df_output)

In [48]:
item_prediction.shape
type(item_prediction)

numpy.ndarray